# Discussion 12A Solution, Fall 2022

In [1]:
import numpy as np
from math import cos, sin, pi, sqrt

In this discussion, we will look at an example of how the SVD and pseudoinverse can be used to solve for the minimum norm solution. This discussion will also help you prepare for a homework problem you will have on HW12.

For this notebook, we will deal with a graphical example where we are given 2 planes and are asked to find the point on the intersection that is the closest to the origin. We will attempt to use the pseudoinverse as a means of calculating this point. Suppose the equations for the two planes are given as follows:
$$
    x + y - z = 4 \\
    x + y + z = 4
$$


### Part (a)
Given the equations above, __write the system in matrix-vector form__
$$
    A\vec{x} = \vec{b} \ \ \ \ \text{where} \ \ \ \ \vec{x} = 
    \begin{bmatrix}
        x \\ y \\ z
    \end{bmatrix}
$$
What type of system is this and how many solutions exist?

HINT: To create a numpy array for the $2 \times 2$ identity matrix, you would write np.array([[1,0], [0,1]])

In [2]:
A = np.array([[1, 1, -1], [1, 1, 1]])
b = np.array([[4], [4]])

m, n = A.shape

### Part (b)
Now let's calculate the SVD of matrix $A$. We have provided you with some skeleton code for the algorithm for calculating the full SVD. It may be helpful to refer to Algorithm 15 on Note 16: https://eecs16b.org/notes/fa22/note16.pdf#page=8. __Fill in the blanks in the algorithm using the note linked above as a reference.__

HINT: Some helpful numpy functions include [np.linalg.eig](https://numpy.org/doc/stable/reference/generated/numpy.linalg.eig.html), [np.linalg.matrix_rank](https://numpy.org/doc/stable/reference/generated/numpy.linalg.matrix_rank.html)

HINT: To perform matrix multiplication you can use the @ operator (ex: A @ B) and to take the transpose of a matrix use .T (ex: A.T)

In [3]:
def FullSVD(A):
    m, n = A.shape
    r = np.linalg.matrix_rank(A)
    U_r = np.zeros((m, r))
    Sigma_r = np.zeros((r, r))
    V_r = np.zeros((n, r))
    
    eigenvalues, normalized_eigenvectors = np.linalg.eig(A.T @ A)

    # Handles sorting of eigenvalues in non-increasing order
    idx = eigenvalues.argsort()[::-1]   
    eigenvalues = eigenvalues[idx]
    normalized_eigenvectors = normalized_eigenvectors[:,idx]

    for i in range(r):
        sigma_i = sqrt(eigenvalues[i])
        v_i = normalized_eigenvectors[:,i]
        u_i = A @ v_i / sigma_i

        Sigma_r[i][i] = sigma_i
        V_r[:,i] = v_i
        U_r[:,i] = u_i
    
    U = ExtendBasis(U_r)
    V = ExtendBasis(V_r)
    Sigma = np.pad(Sigma_r, [(0, m-r), (0, n-r)])
    
    return (U, Sigma, V)

In [4]:
def ExtendBasis(S):
    Q, R = np.linalg.qr(S, mode='complete')
    return Q 

Run the cell below to solve for the SVD of our $A$ matrix using our defined algorithm:

In [5]:
U, Sigma, V = FullSVD(A)
print("U:")
print(U)
print("Sigma:")
print(Sigma)
print("V:")
print(V)

U:
[[-0.70710678 -0.70710678]
 [-0.70710678  0.70710678]]
Sigma:
[[2.         0.         0.        ]
 [0.         1.41421356 0.        ]]
V:
[[-0.70710678  0.          0.70710678]
 [-0.70710678  0.         -0.70710678]
 [-0.         -1.          0.        ]]


### Part (c)
Recall that the pseudoinverse of a matrix $A$ with a compact SVD of $A = U_r \Sigma_r V_r^\top$ can be written as:
$$
    A^{\dagger} := V_r \Sigma_r^{-1}U_r^{\top}
$$
__Calculate the compact pseudoinverse of $A$ and then solve for the minimum norm solution for $\vec{x}$.__

HINT: You may find the following function useful: https://numpy.org/doc/stable/reference/generated/numpy.linalg.inv.html

In [6]:
r = np.linalg.matrix_rank(A)
A_dagger = V[:,:r] @ np.linalg.inv(Sigma[:r,:r]) @ U[:,:r].T 
min_norm = A_dagger @ b
print("Pseudoinverse: \n", A_dagger)
print("Min Norm Solution: \n", min_norm)

Pseudoinverse: 
 [[ 0.25  0.25]
 [ 0.25  0.25]
 [ 0.5  -0.5 ]]
Min Norm Solution: 
 [[ 2.0000000e+00]
 [ 2.0000000e+00]
 [-4.4408921e-16]]


Let's check our work by comparing our solution with the solution that numpy calculates.

In [7]:
A_dagger_np = np.linalg.pinv(A)
min_norm_np = A_dagger_np @ b

print("Expected pseudoinverse: \n", A_dagger_np)
print("Actual pseudoinverse: \n", A_dagger)
print("Expected min norm solution: \n", min_norm_np)
print("Actual min norm solution: \n", min_norm)

Expected pseudoinverse: 
 [[ 0.25  0.25]
 [ 0.25  0.25]
 [-0.5   0.5 ]]
Actual pseudoinverse: 
 [[ 0.25  0.25]
 [ 0.25  0.25]
 [ 0.5  -0.5 ]]
Expected min norm solution: 
 [[2.0000000e+00]
 [2.0000000e+00]
 [8.8817842e-16]]
Actual min norm solution: 
 [[ 2.0000000e+00]
 [ 2.0000000e+00]
 [-4.4408921e-16]]


### Part (d)
Recall that for a wide matrix $A \in \mathbb{R}^{m \times n}$ where $m \leq n$ and rank($A$) = $m$ ($A$ has full row rank), the pseudoinverse can be represented as
$$
    A^\dagger = A^\top (AA^\top)^{-1}
$$
__Use this definition of the pseudoinverse to solve for the minimum norm solution and show that they are the same.__

HINT: You may find the following function useful: https://numpy.org/doc/stable/reference/generated/numpy.linalg.inv.html. Also recall that matrix multiplication uses the operator @.

In [8]:
A_dagger_full_rank = A.T @ np.linalg.inv(A @ A.T)
min_norm_full_rank = A_dagger_full_rank @ b
print("Pseudoinverse: \n", A_dagger_full_rank)
print("Min Norm Solution: \n ", min_norm_full_rank)

Pseudoinverse: 
 [[ 0.25  0.25]
 [ 0.25  0.25]
 [-0.5   0.5 ]]
Min Norm Solution: 
  [[2.]
 [2.]
 [0.]]


### Part (e)
Suppose we tried to use the following form of the pseudoinverse to calculate the minimum norm solution for our problem:
$$
    A^\dagger =  (A^\top A)^{-1}A^\top
$$
__Why does or doesn't this form of the pseudoinverse work for solving our problem? Verify your response below.__

In [9]:
A_dagger_ls = np.linalg.inv(A.T @ A) @ A.T
min_norm_ls = A_dagger_ls @ b
print("Pseudoinverse: \n", A_dagger_ls)
print("Min Norm Solution: \n ", min_norm_ls)

LinAlgError: Singular matrix

If you plugged in the pseudoinverse formula and attempted to calculate it, you should've run into a "LinAlgError: Singular matrix" error. Essentially this error states that you have a square matrix that doesn't have an inverse. In this case, $A\top A$ is not invertible, since we only have a rank 2 $A$ matrix so $A\top A$ will not be full rank. Therefore it is not invertible and this form of the pseudoinverse wouldn't work for an underdetermined/wide system. This formula solves the least squares problem where we have an overdetermined system and are trying to solve for the best estimate of $\vec{b}$.

Contributors:
- Oliver Yu